# Priprema podataka u KH formatu
### Podaci kopirani sa MAID stranice u kombinaciji I=+,-,0
https://maid.kph.uni-mainz.de/maid2007/mult.html

In [166]:
import pandas as pd
import numpy as np

In [167]:
cd Q2=1_Iso

/home/rifat/Dropbox/ElectroP/podaci/Q2=1_Iso


In [168]:
E0p = pd.read_csv('E0+.dat', sep=r'\s+') #ucitavam samo E0+ radi zaglavlja

In [169]:
# generate null multipoles: E0-, M0+, M0-,S0-, E1-
empty = np.zeros(shape=E0p.shape)
df_zeros = pd.DataFrame(empty, columns=E0p.columns)
df_zeros['W(MeV)'] = E0p['W(MeV)']
df_zeros.to_csv('E0-.dat', sep='\t')
df_zeros.to_csv('M0+.dat', sep='\t')
df_zeros.to_csv('M0-.dat', sep='\t')
df_zeros.to_csv('S0-.dat', sep='\t')
df_zeros.to_csv('E1-.dat', sep='\t')
df_zeros.head()


,W(MeV),Re(0),Im(0),Re(+),Im(+),Re(-),Im(-),wlab(MeV),wcm(MeV)
0,1080.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1085.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1090.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1095.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [170]:
#Read all multipoles and append to one dataframe
import glob 
# 1. Declare directory where csv files are stored
directory = "*.dat"

# 2. Create empty dataframe
df = pd.DataFrame()
#df['W(MeV)'] = E0p['W(MeV)']
  
# 3. Iterate through each csv file in the directory, read it into a
# temporary dataframe and then append this to dataframe to the main
# dataframe

for file in sorted(glob.glob(directory)):
    temp_df = pd.read_csv(file, sep=r'\s+')
    temp_df['MP'] = str(file)[:3]  #add multipole name into column
    temp_df['L'] = str(file)[1]  #add value of L, for sorting later
    df = df.append(temp_df)
    
df.head(200)    


,W(MeV),Re(0),Im(0),Re(+),Im(+),Re(-),Im(-),wlab(MeV),wcm(MeV),MP,L
0,1080.0,0.776,0.015,3.588,0.039,2.003,0.056,685.3,-330.5,E0+,0
1,1085.0,0.774,0.038,3.512,0.097,2.025,0.137,691.1,-324.0,E0+,0
2,1090.0,0.772,0.052,3.439,0.134,2.046,0.185,696.9,-317.5,E0+,0
3,1095.0,0.770,0.062,3.368,0.162,2.065,0.222,702.7,-311.1,E0+,0
4,1100.0,0.769,0.068,3.302,0.176,2.084,0.241,708.6,-304.7,E0+,0
...,...,...,...,...,...,...,...,...,...,...,...
95,1555.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,E0-,0
96,1560.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,E0-,0
97,1565.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,E0-,0
98,1570.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,E0-,0


In [171]:
# Izdvajam samo kolone koje trebaju i pravim tri dataframe prema vrijednstima I=0,+,-
## sortiram po energiji, zatim po L i onda po redu E,M,S
Iso_0 = df.sort_values(['W(MeV)','L','MP'])[['W(MeV)','Re(0)','Im(0)','MP']]
Iso_p = df.sort_values(['W(MeV)','L','MP'])[['W(MeV)','Re(+)','Im(+)','MP']]
Iso_m = df.sort_values(['W(MeV)','L','MP'])[['W(MeV)','Re(-)','Im(-)','MP']]

In [172]:
# prebacujem E0+ i E0- u isti red. Tako i za ostale multopole, kako bih dobio u istom redu isti L i isti Multipol
Iso0 = pd.DataFrame([y.values.ravel() for x , y in Iso_0.groupby(np.arange(len(Iso_0))//2)])
Isop = pd.DataFrame([y.values.ravel() for x , y in Iso_p.groupby(np.arange(len(Iso_p))//2)])
Isom = pd.DataFrame([y.values.ravel() for x , y in Iso_m.groupby(np.arange(len(Iso_m))//2)])

In [173]:
# Popravljanje imena kolona - prethodni korak je napravio duple nazive
Iso0.columns = ['W(MeV)','Re(0)+','Im(0)+','MP','W(MeV)_','Re(0)-','Im(0)-','MP']
Isop.columns = ['W(MeV)','Re(+)+','Im(+)+','MP','W(MeV)_','Re(+)-','Im(+)-','MP']
Isom.columns = ['W(MeV)','Re(-)+','Im(-)+','MP','W(MeV)_','Re(-)-','Im(-)-','MP']

In [174]:
# Izdvajanje samo potrebnih kolona za KH format
Iso0_KH = Iso0[['W(MeV)','Re(0)+','Im(0)+','Re(0)-','Im(0)-']]
Isop_KH = Isop[['W(MeV)','Re(+)+','Im(+)+','Re(+)-','Im(+)-']]
Isom_KH = Isom[['W(MeV)','Re(-)+','Im(-)+','Re(-)-','Im(-)-']]

In [175]:
Iso0_KH.head(10)
Isom_KH.shape

(1800, 5)

In [176]:
# Vrijendost energije da pise samo na prvom mjestu, u prvom redu
Iso0_KH.loc[Iso0_KH['W(MeV)'].duplicated(), 'W(MeV)'] = '      ' 
Isop_KH.loc[Isop_KH['W(MeV)'].duplicated(), 'W(MeV)'] = '      ' 
Isom_KH.loc[Isom_KH['W(MeV)'].duplicated(), 'W(MeV)'] = '      '

/home/rifat/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [188]:
# Provjera formata:
print(Isom_KH.head())
print(Isop_KH.head())
print(Iso0_KH.head())

   W(MeV)  Re(-)+  Im(-)+  Re(-)-  Im(-)-
0    1080   2.003   0.056   0.000     0.0
1           0.000   0.000   0.000     0.0
2           2.497   0.013   0.000     0.0
3           0.171   0.000   0.000     0.0
4          -0.933  -0.001  -0.992     0.0
   W(MeV)  Re(+)+  Im(+)+  Re(+)-  Im(+)-
0    1080   3.588   0.039   0.000   0.000
1           0.000   0.000   0.000   0.000
2          -1.078   0.052   0.000   0.000
3          -0.140   0.000   0.000   0.000
4           1.358   0.002  -1.305   0.001
   W(MeV)  Re(0)+  Im(0)+  Re(0)-  Im(0)-
0    1080   0.776   0.015   0.000     0.0
1           0.000   0.000   0.000     0.0
2          -1.740  -0.035   0.000     0.0
3           0.006   0.000   0.000     0.0
4           0.088   0.000  -0.355     0.0


In [178]:
# Snimanje konacnog rezultata u datoteku:
Iso0_KH.to_csv('KH_format/Iso=0.dat', sep='\t', index=False, float_format='%9.3f') 
Isop_KH.to_csv('KH_format/Iso=+.dat', sep='\t', index=False, float_format='%9.3f') 
Isom_KH.to_csv('KH_format/Iso=-.dat', sep='\t', index=False, float_format='%9.3f') 

In [179]:
nrofen = Iso0.nunique()[0] #Broj energija
print(nrofen)

100


### Provjera jesam li dobro downloadovao sam MAID stranice
Upoređujem vrijednosti koje sam skinuo za I=+,-,0 i I=p(1/2), n(1/2), 3/2.
Ako jesam, jednakosti iz jednacina bi trebale vrijediti!

$$ A^0 + A^+ = A_p^{1/2}+ 2/3 A^{3/2} $$
$$ \sqrt(2)(A^- + A^0) = A_p^{1/2}- 1/3 A^{3/2}$$

In [180]:
E0p_I = pd.read_csv('E0+.dat', sep=r'\s+') #I = +,-,0

In [181]:
cd ..

/home/rifat/Dropbox/ElectroP/podaci


In [182]:
E0p_312= pd.read_csv('Q2=1/E0+.dat', sep=r'\s+')  # I= p(1/2),n(1/2), 3/2

In [183]:
print(E0p_312.columns)
print(E0p_I.columns)

Index(['W(MeV)', 'Re(p1/2)', 'Im(p1/2)', 'Re(n1/2)', 'Im(n1/2)', 'Re(3/2)',
       'Im(3/2)', 'lab(MeV)', 'cm(MeV)'],
      dtype='object')
Index(['W(MeV)', 'Re(0)', 'Im(0)', 'Re(+)', 'Im(+)', 'Re(-)', 'Im(-)',
       'wlab(MeV)', 'wcm(MeV)'],
      dtype='object')


In [184]:
#Projvera jednakosti:
L1 = E0p_I['Re(0)'] + E0p_I['Re(+)'] 
D1 = E0p_312['Re(p1/2)']+ 2*E0p_312['Re(3/2)']/3

L2 = np.sqrt(2)* (E0p_I['Re(-)'] + E0p_I['Re(0)'])
D2 = np.sqrt(2)*(E0p_312['Re(p1/2)']- E0p_312['Re(3/2)']/3)

In [185]:
#ispis provjere jednakosti gornjih jednacina
pd.DataFrame([D1, L1, D2, L2], index=['Eq1_left','Eq1_right','Eq2_left','Eq2_right'])

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
Eq1_left,4.363667,4.286333,4.210667,4.138333,4.071000,4.005333,3.941667,3.880000,3.822000,3.765333,...,1.061000,0.674000,0.343667,0.079000,-0.121000,-0.261667,-0.350000,-0.397667,-0.412000,-0.403333
Eq1_right,4.364000,4.286000,4.211000,4.138000,4.071000,4.006000,3.941000,3.880000,3.822000,3.766000,...,1.061000,0.674000,0.344000,0.079000,-0.121000,-0.261000,-0.350000,-0.398000,-0.412000,-0.404000
Eq2_left,3.929628,3.958855,3.984782,4.008353,4.034751,4.057850,4.079535,4.099805,4.119604,4.138460,...,2.429619,1.865348,1.379801,0.985707,0.681651,0.462919,0.319612,0.236645,0.206475,0.213075
Eq2_right,3.930099,3.958384,3.985254,4.009295,4.034751,4.058793,4.080006,4.099805,4.119604,4.137989,...,2.429619,1.865348,1.380272,0.985707,0.681651,0.463862,0.319612,0.237588,0.206475,0.212132


Prethodni dataframe pokazuje da su desne i lijeve strane odgovarajućih jednačina jednake!!!